In [1]:
!pip install transformers

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [33]:
URL = "https://www.ibm.com/topics/deep-learning"

In [34]:
r = requests.get(URL)

In [35]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [36]:
ARTICLE

'Deep learning is a subset of machine learning that uses multi-layered neural networks, called deep neural networks, to simulate the complex decision-making power of the human brain. Some form of deep learning powers most of the artificial intelligence (AI) in our lives today. By strict definition, a\xa0deep neural network, or DNN, is a neural network with three or more layers. In practice, most DNNs have many more layers. DNNs are trained on large amounts of data to identify and classify phenomena, recognize patterns and relationships, evaluate posssibilities, and make predictions and decisions. While a single-layer neural network can make useful, approximate predictions and decisions, the additional layers in a deep neural network help refine and optimize those outcomes for greater accuracy. Deep learning drives many\xa0applications and services that improve\xa0automation, performing analytical and physical tasks without human intervention. It lies behind everyday products and servic

In [37]:
max_chunk = 500

In [38]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [39]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [40]:
len(chunks)

3

# 4. Summarize Text

In [41]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [42]:
res[0]

{'summary_text': ' Deep learning is a subset of machine learning that uses multi-layered neural networks to simulate the complex decision-making power of the human brain . Some form of deep learning powers most of the artificial intelligence in our lives today . Deep learning drives many applications and services that improve\xa0automation, performing analytical and physical tasks without human intervention .'}

In [43]:
' '.join([summ['summary_text'] for summ in res])

' Deep learning is a subset of machine learning that uses multi-layered neural networks to simulate the complex decision-making power of the human brain . Some form of deep learning powers most of the artificial intelligence in our lives today . Deep learning drives many applications and services that improve\xa0automation, performing analytical and physical tasks without human intervention .  In contrast, unsupervised learning doesn’t require labeled datasets, and instead, it detects patterns in the data . Reinforcement learning is a process in which a model learns to become more accurate for performing an action in an environment based on feedback in order to maximize the reward . For a deeper dive on the nuanced differences between the different technologies, see "AI vs. Machine Learning vs.  Deep Learning: What’s the Difference? "  IBM watsonx is a portfolio of business-ready tools, applications and solutions, designed to reduce the costs and hurdles of AI adoption . Virtual assist

In [44]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [45]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)